In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
#We read the Toronto dataset created earlier and pick out the coordinates for Central Toronto to get our starint point
toronto = pd.read_csv(r"Toronto.csv",index_col=0) 

#create coordinates longitude and latitude
longitude = toronto.Longitude[0]
latitude = toronto.Latitude[0]
print(longitude,latitude)

toronto

-79.3887901 43.7280205


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,"Lawrence Park, Davisville North, North Toronto...",43.728020,-79.388790
1,M4W,Downtown Toronto,"Rosedale, St. James Town, Cabbagetown, Church ...",43.679563,-79.377529
2,M4E,East Toronto,"The Beaches, The Danforth West, Riverdale, Ind...",43.676357,-79.293031
3,M4B,East York,"Parkview Hill, Woodbine Gardens, Woodbine Heig...",43.706397,-79.309937
4,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores, ...",43.605647,-79.501321
5,M7R,Mississauga,Canada Post Gateway Processing Centre,43.636966,-79.615819
6,M2H,North York,"Hillcrest Village, Fairview, Henry Farm, Oriol...",43.803762,-79.363452
7,M1B,Scarborough,"Malvern, Rouge, Rouge Hill, Port Union, Highla...",43.806686,-79.194353
8,M6H,West Toronto,"Dufferin, Dovercourt Village, Little Portugal,...",43.669005,-79.442259
9,M6C,York,"Humewood-Cedarvale, Caledonia-Fairbanks, Del R...",43.693781,-79.428191


We see that Toronto is around the coordinates Latitude 43 and Longitude -79. We create a a map of Toronto with neighborhoods superimposed on top.

In [3]:
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=11)

for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], 
                                           toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define Foursquare Credentials and Version

In [4]:
CLIENT_ID = 'YD10VKEJURAHPXLUPGAJRGBHNMMUMJ5W3MBTRHFSU23LEA5Q' # your Foursquare ID
CLIENT_SECRET = 'UXS10GHOFJYIXWZ1W5LIW3SWQ5HBXJ4ZLF2RJ1315XBBQKGS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YD10VKEJURAHPXLUPGAJRGBHNMMUMJ5W3MBTRHFSU23LEA5Q
CLIENT_SECRET:UXS10GHOFJYIXWZ1W5LIW3SWQ5HBXJ4ZLF2RJ1315XBBQKGS


#### Let's explore the second borough in our dataframe which is Downtown Toronto.

We get the coordinates for Downtown Toronto.

In [5]:
borough_latitude = toronto.loc[1, 'Latitude'] # neighborhood latitude value
borough_longitude = toronto.loc[1, 'Longitude'] # neighborhood longitude value

borough_name = toronto.loc[1, 'Borough'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(borough_name, 
                                                               borough_latitude, 
                                                               borough_longitude))

Latitude and longitude values of Downtown Toronto are 43.6795626, -79.37752940000001.


#### Let's get the top 10 venues that are in Downtown Toronto within a radius of 500 meters.

We create the GET request URL named url.

In [6]:
LIMIT = 10 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    borough_latitude, 
    borough_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=YD10VKEJURAHPXLUPGAJRGBHNMMUMJ5W3MBTRHFSU23LEA5Q&client_secret=UXS10GHOFJYIXWZ1W5LIW3SWQ5HBXJ4ZLF2RJ1315XBBQKGS&v=20180605&ll=43.6795626,-79.37752940000001&radius=500&limit=10'

We send the GET request and look at the result.

In [7]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5efdc14721cdee79b05758b5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Rosedale',
  'headerFullLocation': 'Rosedale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.6840626045, 'lng': -79.37131878274371},
   'sw': {'lat': 43.675062595499995, 'lng': -79.38374001725632}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4aff2d47f964a520743522e3',
       'name': 'Rosedale Park',
       'location': {'address': '38 Scholfield Ave.',
        'crossStreet': 'at Edgar Ave.',
        'lat': 43.68232820227814,
        'lng': -79.37893434347683,
        'labeledLatLngs': [{'label': 'd

We  borrow the get_category_type function from the Foursquare lab.

In [8]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe. We see that Foursquare have returned 4 venues.

In [9]:
import warnings
warnings.filterwarnings('ignore')

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

nearby_venues

4 venues were returned by Foursquare.


,name,categories,lat,lng
0,Rosedale Park,Playground,43.682328,-79.378934
1,Whitney Park,Park,43.682036,-79.373788
2,Alex Murray Parkette,Park,43.678300,-79.382773
3,Milkman's Lane,Trail,43.676352,-79.373842


Like in the lab we create a function to repeat the same process to all the boroughs in Toronto.

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each borough and create a new dataframe called toronto_venues.

In [11]:
toronto_venues = getNearbyVenues(names=toronto['Borough'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )


Central Toronto
Downtown Toronto
East Toronto
East York
Etobicoke
Mississauga
North York
Scarborough
West Toronto
York


Let's check the size of the resulting dataframe.

In [12]:
print(toronto_venues.shape)
toronto_venues.head()

(60, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Central Toronto,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Central Toronto,43.728020,-79.388790,The Photo School – Toronto,43.730429,-79.388767,Photography Studio
2,Central Toronto,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
3,Central Toronto,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
4,Downtown Toronto,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground


We see that there are only 4 boroughs that return 10 results within the radius. 

In [13]:
toronto_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,4,4,4,4,4,4
Downtown Toronto,4,4,4,4,4,4
East Toronto,4,4,4,4,4,4
East York,10,10,10,10,10,10
Etobicoke,10,10,10,10,10,10
Mississauga,10,10,10,10,10,10
North York,4,4,4,4,4,4
Scarborough,1,1,1,1,1,1
West Toronto,10,10,10,10,10,10


We see, that there are 40 unique categories in the boroughs.

In [14]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 40 uniques categories.


In [15]:
# one hot encoding
onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Borough'] = toronto_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Borough,American Restaurant,Bakery,Bank,Bar,Breakfast Spot,Brewery,Burrito Place,Bus Line,Café,Coffee Shop,Dog Run,Fast Food Restaurant,Field,Fried Chicken Joint,Gastropub,Golf Course,Grocery Store,Gym / Fitness Center,Health Food Store,Hockey Arena,Hotel,Intersection,Liquor Store,Mediterranean Restaurant,Middle Eastern Restaurant,Music Venue,Neighborhood,Park,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Pool,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Swim School,Trail
0,Central Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,Central Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,Central Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,Central Toronto,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Downtown Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


Next, let's group rows by borough and by taking the mean of the frequency of occurrence of each category

In [16]:
grouped = onehot.groupby('Borough').mean().reset_index()
grouped

,Borough,American Restaurant,Bakery,Bank,Bar,Breakfast Spot,Brewery,Burrito Place,Bus Line,Café,Coffee Shop,Dog Run,Fast Food Restaurant,Field,Fried Chicken Joint,Gastropub,Golf Course,Grocery Store,Gym / Fitness Center,Health Food Store,Hockey Arena,Hotel,Intersection,Liquor Store,Mediterranean Restaurant,Middle Eastern Restaurant,Music Venue,Neighborhood,Park,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Pool,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Swim School,Trail
0,Central Toronto,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.25,0.0,0.0,0.25,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.25,0.000000
1,Downtown Toronto,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.50,0.0,0.0,0.00,0.0,0.25,0.00,0.00,0.0,0.0,0.0,0.00,0.250000
2,East Toronto,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.25,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.25,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.25,0.0,0.0,0.0,0.00,0.250000
3,East York,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.00,0.0,0.0,0.00,0.1,0.000000,0.0,0.1,0.00,0.0,0.1,0.00,0.000000,0.0,0.1,0.0,0.00,0.0,0.0,0.00,0.00,0.1,0.1,0.00,0.2,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.000000
4,Etobicoke,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.00,0.1,0.0,0.00,0.1,0.000000,0.1,0.0,0.00,0.0,0.0,0.00,0.000000,0.0,0.0,0.1,0.00,0.0,0.0,0.00,0.00,0.0,0.1,0.00,0.1,0.00,0.00,0.00,0.1,0.0,0.1,0.00,0.000000
5,Mississauga,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.00,0.0,0.2,0.00,0.0,0.000000,0.1,0.0,0.00,0.0,0.0,0.00,0.000000,0.2,0.0,0.0,0.10,0.1,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.1,0.0,0.00,0.000000
6,North York,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.25,0.0,0.000000,0.0,0.0,0.25,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.25,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.25,0.00,0.0,0.0,0.0,0.00,0.000000
7,Scarborough,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,1.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.000000
8,West Toronto,0.0,0.2,0.1,0.1,0.0,0.1,0.0,0.00,0.1,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.1,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.1,0.1,0.00,0.00,0.0,0.1,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.000000
9,York,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.333333,0.0,0.0,0.00,0.0,0.0,0.00,0.333333,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.333333


Let's print each borough along with the top 5 most common venues

In [17]:
num_top_venues = 5

for bor in grouped['Borough']:
    print("----"+bor+"----")
    temp = grouped[grouped['Borough'] == bor].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
                 venue  freq
0   Photography Studio  0.25
1          Swim School  0.25
2             Bus Line  0.25
3                 Park  0.25
4  American Restaurant  0.00


----Downtown Toronto----
          venue  freq
0          Park  0.50
1         Trail  0.25
2    Playground  0.25
3      Pharmacy  0.00
4  Liquor Store  0.00


----East Toronto----
               venue  freq
0              Trail  0.25
1  Health Food Store  0.25
2                Pub  0.25
3       Neighborhood  0.25
4           Pharmacy  0.00


----East York----
                  venue  freq
0           Pizza Place   0.2
1  Fast Food Restaurant   0.1
2                  Bank   0.1
3          Intersection   0.1
4        Breakfast Spot   0.1


----Etobicoke----
                 venue  freq
0  American Restaurant   0.1
1                 Café   0.1
2               Bakery   0.1
3             Pharmacy   0.1
4  Fried Chicken Joint   0.1


----Mississauga----
                       venue  freq
0      

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Borough_venues_sorted = pd.DataFrame(columns=columns)
Borough_venues_sorted['Borough'] = grouped['Borough']

for ind in np.arange(grouped.shape[0]):
    Borough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

Borough_venues_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Swim School,Park,Bus Line,Photography Studio,Grocery Store,Golf Course,Gastropub,Fried Chicken Joint,Field,Fast Food Restaurant
1,Downtown Toronto,Park,Trail,Playground,Coffee Shop,Grocery Store,Golf Course,Gastropub,Fried Chicken Joint,Field,Fast Food Restaurant
2,East Toronto,Trail,Pub,Neighborhood,Health Food Store,Dog Run,Grocery Store,Golf Course,Gastropub,Fried Chicken Joint,Field
3,East York,Pizza Place,Pet Store,Bank,Gastropub,Breakfast Spot,Intersection,Pharmacy,Gym / Fitness Center,Fast Food Restaurant,Dog Run
4,Etobicoke,American Restaurant,Seafood Restaurant,Bakery,Restaurant,Fried Chicken Joint,Pizza Place,Liquor Store,Pharmacy,Fast Food Restaurant,Café
5,Mississauga,Coffee Shop,Hotel,Burrito Place,Fried Chicken Joint,Mediterranean Restaurant,Middle Eastern Restaurant,American Restaurant,Sandwich Place,Brewery,Breakfast Spot
6,North York,Golf Course,Mediterranean Restaurant,Pool,Dog Run,Trail,Coffee Shop,Grocery Store,Gastropub,Fried Chicken Joint,Field
7,Scarborough,Fast Food Restaurant,Trail,Swim School,Gym / Fitness Center,Grocery Store,Golf Course,Gastropub,Fried Chicken Joint,Field,Dog Run
8,West Toronto,Bakery,Pharmacy,Brewery,Middle Eastern Restaurant,Music Venue,Café,Grocery Store,Bar,Bank,Burrito Place
9,York,Trail,Field,Hockey Arena,Sandwich Place,Gym / Fitness Center,Golf Course,Gastropub,Fried Chicken Joint,Seafood Restaurant,Fast Food Restaurant


#### Cluster Boroughs like in the lab.

Run k-means to cluster the neighborhood into 5 clusters.

In [20]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 0, 0, 0, 0, 3, 4, 0, 2])

In [21]:
# add clustering labels
Borough_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged = toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(Borough_venues_sorted.set_index('Borough'), on='Borough')

merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,"Lawrence Park, Davisville North, North Toronto...",43.728020,-79.388790,1,Swim School,Park,Bus Line,Photography Studio,Grocery Store,Golf Course,Gastropub,Fried Chicken Joint,Field,Fast Food Restaurant
1,M4W,Downtown Toronto,"Rosedale, St. James Town, Cabbagetown, Church ...",43.679563,-79.377529,1,Park,Trail,Playground,Coffee Shop,Grocery Store,Golf Course,Gastropub,Fried Chicken Joint,Field,Fast Food Restaurant
2,M4E,East Toronto,"The Beaches, The Danforth West, Riverdale, Ind...",43.676357,-79.293031,0,Trail,Pub,Neighborhood,Health Food Store,Dog Run,Grocery Store,Golf Course,Gastropub,Fried Chicken Joint,Field
3,M4B,East York,"Parkview Hill, Woodbine Gardens, Woodbine Heig...",43.706397,-79.309937,0,Pizza Place,Pet Store,Bank,Gastropub,Breakfast Spot,Intersection,Pharmacy,Gym / Fitness Center,Fast Food Restaurant,Dog Run
4,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores, ...",43.605647,-79.501321,0,American Restaurant,Seafood Restaurant,Bakery,Restaurant,Fried Chicken Joint,Pizza Place,Liquor Store,Pharmacy,Fast Food Restaurant,Café
5,M7R,Mississauga,Canada Post Gateway Processing Centre,43.636966,-79.615819,0,Coffee Shop,Hotel,Burrito Place,Fried Chicken Joint,Mediterranean Restaurant,Middle Eastern Restaurant,American Restaurant,Sandwich Place,Brewery,Breakfast Spot
6,M2H,North York,"Hillcrest Village, Fairview, Henry Farm, Oriol...",43.803762,-79.363452,3,Golf Course,Mediterranean Restaurant,Pool,Dog Run,Trail,Coffee Shop,Grocery Store,Gastropub,Fried Chicken Joint,Field
7,M1B,Scarborough,"Malvern, Rouge, Rouge Hill, Port Union, Highla...",43.806686,-79.194353,4,Fast Food Restaurant,Trail,Swim School,Gym / Fitness Center,Grocery Store,Golf Course,Gastropub,Fried Chicken Joint,Field,Dog Run
8,M6H,West Toronto,"Dufferin, Dovercourt Village, Little Portugal,...",43.669005,-79.442259,0,Bakery,Pharmacy,Brewery,Middle Eastern Restaurant,Music Venue,Café,Grocery Store,Bar,Bank,Burrito Place
9,M6C,York,"Humewood-Cedarvale, Caledonia-Fairbanks, Del R...",43.693781,-79.428191,2,Trail,Field,Hockey Arena,Sandwich Place,Gym / Fitness Center,Golf Course,Gastropub,Fried Chicken Joint,Seafood Restaurant,Fast Food Restaurant


Visualization of the resulting clusters

In [22]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], 
                                  merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters